In [1]:
import os
import glob

#from tqdm import tqdm
import numpy as np
import scipy.ndimage
import scipy.misc
import pandas as pd
import pickle
from IPython.display import clear_output

import keras
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers import Concatenate
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.models import Model

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
# import the necessary packages
from keras.preprocessing import image as image_utils
from keras.utils import plot_model
from keras import backend as K


from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

from scipy import stats

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import random as rn


# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess) # reference: https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer



Using TensorFlow backend.
/l/anaconda3-5.2.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
IMG_SIZE = (256, 256)
IN_SHAPE = (*IMG_SIZE, 3)
BATCH_SIZE = 64

In [3]:
openImg_path = '/nfs/juhu/data/rakhasan/bystander-detection/google-img-db/'
survey_path='/nfs/juhu/data/rakhasan/bystander-detection/pilot-study2/'
survey_photo_path = survey_path+'/photos/'

model_output_path = '/nfs/juhu/data/rakhasan/bystander-detection/code-repos/notebooks/model-output/'

print('loading features.')
feature_df = pickle.load(open(os.path.join(survey_path, 'features-df.pkl'), 'rb'))

loading features.


In [5]:
feature_df_resnet = pickle.load(open(os.path.join(survey_path, 'features-df-resnet.pkl'), 'rb'))
#feature_df = feature_df.loc[feature_df_resnet.index]
# feature_df['resnet_feature'] = feature_df_resnet.resnet_feature
# feature_df.shape

In [8]:
list(feature_df_resnet.columns)[:20]

['was_aware_num',
 'posing_num',
 'comfort_num',
 'will_num',
 'photographer_intention_num',
 'replacable_num',
 'photo_place_num',
 'subject_bystander_num',
 'label',
 'cropped_photo',
 'resized_cropped_img',
 'person_distance',
 'person_size',
 'num_people',
 'Human_eye',
 'Human_beard',
 'Human_mouth',
 'Human_body',
 'Human_foot',
 'Human_leg']

In [63]:
from keras.layers import GlobalAveragePooling2D


dic = dict()
sess = tf.Session()
with sess.as_default():
    for index, row in feature_df_resnet.iterrows():
        tensor = tf.convert_to_tensor(
            row.resnet_feature,
            dtype=None,
            name=None,
            preferred_dtype=None
        )

        p=GlobalAveragePooling2D()(tensor)
        dic[index] = p.eval()

In [71]:
dic['1'][0]

array([2.7821752e-01, 1.9637873e+00, 1.2800893e-01, ..., 9.8553133e-01,
       1.8732380e-03, 1.9474829e-02], dtype=float32)

In [72]:
#pickle.dump(dic, open(survey_path+'resnet_features_averaged.pkl','wb'))

In [ ]:
from tqdm import tqdm
visual_features = pd.DataFrame()
for i in tqdm(range(131072)):
    visual_features['resnet_feat_{}'.format(i)] = feature_df_resnet.apply(lambda row: row.res_flat[i], axis=1)

In [ ]:
feature_df_resnet['res_flat']=feature_df_resnet.apply(lambda row: row.resnet_feature.flatten(), axis=1)

In [ ]:
'''Add resnet feature in the feature dataframe'''
#feature_df['resnet_feature'] = resnet_model.predict(
    #np.array(list(feature_df.resized_cropped_img),batch_size=BATCH_SIZE), axis=1)
#feature_df['resnet_feature'] = feature_df.apply(lambda row: row.resnet_feature.flatten(), axis=1)
feature_df['resnet_feature_zscore'] = feature_df.apply(lambda row: row.resnet_feature.apply(), axis=1)